In [56]:
#V0.12
#Changing clf from Logistic-Reg to RandomforestClassifier

import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [57]:
#read csv
train = pd.read_csv('/home/centos/My notebooks/Titanic dataset/train.csv')
X_kaggle = pd.read_csv(r'/home/centos/My notebooks/Titanic dataset/test.csv')

print train.shape
print X_kaggle.shape

(891, 12)
(418, 11)


In [58]:
#basic info
train.info()
train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [59]:
#pd.DataFrame(StandardScaler().fit_transform(train[['Fare']]),columns=['a'])

In [60]:
#Function transformers
func_transformer_get_numeric_features = FunctionTransformer(lambda x: x.select_dtypes(['float64','int64']),validate=False)
func_transformer_get_text_features = FunctionTransformer(lambda x: x.select_dtypes(['object']),validate=False)

#Label encoder works on each column
#func_transformer_labelencoder = FunctionTransformer(lambda x: x.apply(LabelEncoder().fit_transform, axis=0),validate=False)
func_transformer_labelencoder = FunctionTransformer(lambda x: x.apply(lambda x: pd.factorize(x)[0], axis=0,broadcast=True),validate=False)
#
func_transformer_convert_pd_np = FunctionTransformer(lambda x: x.as_matrix(),validate=False)
#Onehot works on np matrix
#func_transformer_onehotencoder = FunctionTransformer(lambda x: OneHotEncoder(sparse=False).fit_transform(x),validate=False)
func_transformer_onehotencoder = FunctionTransformer(lambda x: pd.get_dummies(x),validate=False)
#For debugging
func_print_stuff = FunctionTransformer(lambda x: func_print(x), validate=False)
def func_print(x):
    print 'x.shape ' ,  x.shape
    
    #try:
        #print 'x.columns ' ,  x.head()
    #except:
    #    pass
    return x

#Impute features
func_transformer_imputer = FunctionTransformer(lambda x: x.apply(lambda x: func_func_transformer_imputer(x),axis=0,broadcast=True),validate=False)
def func_func_transformer_imputer(x):    
    #print '!!!x.dtype ', x.dtype
    if x.dtype == 'object':
        x_ret = x.fillna(x.value_counts().index[0])
        #print x.value_counts().index[0]
    elif x.dtype in ['float64','int64']:
        x_ret = x.fillna(x.median()) #,inplace=True
        #print x.max(), x.median()
    return x_ret

func_transformer_engineer_text = FunctionTransformer(lambda x: func_func_transformer_engineer_text(x),validate=False)
def func_func_transformer_engineer_text(x):
    x_ret = pd.DataFrame()    
    x_ret['Sirname_'] = x.Name.apply(lambda x: (x.split(',')[0]))
    x_ret['Title_'] = x.Name.apply(lambda x: x.split(',')[1]).apply(lambda x: x.split('.')[0])
    temp = x.Ticket.map(lambda x: x.replace('/','').replace(',','').replace('.','').replace(' ','').strip())
    x_ret['Ticket_'] = temp.map(lambda x: 'XXX' if x.isdigit() else ''.join(list(filter(lambda y: not y.isdigit(),x))) )
    return x_ret
    #return np.ones(x.shape)
    
func_transformer_engineer_numeric = FunctionTransformer(lambda x: func_func_transformer_engineer_numeric(x),validate=False)
def func_func_transformer_engineer_numeric(x):
    x_ret = pd.DataFrame()
    x_ret['FamilySize_'] = x.SibSp + x.Parch
    return x_ret
    
func_transformer_drop_manual = FunctionTransformer(lambda x: x.drop(['Name','PassengerId','Ticket'], axis=1))

In [61]:
#1st cut model (combine all)
combine = pd.concat([train,X_kaggle])

#combine_dropna = pd.concat([train.dropna(),X_kaggle.dropna()])
#print 'train.dropna().shape', train.dropna().shape
#print 'X_kaggle.dropna().shape', X_kaggle.dropna().shape
#X = combine_dropna.drop(['Survived','Name','PassengerId','Ticket'], axis=1)
#y = combine_dropna['Survived'] #.values.reshape(-1,1)

#X = combine.drop(['Survived','Name','PassengerId','Ticket'], axis=1)
X = combine.drop(['Survived'],axis=1)
y = combine['Survived'] 

print 'X.shape', X.shape
print 'y.shape', y.shape

#Feature transformations:-
pipe = Pipeline([
    
    ('feature_union1', FeatureUnion(
        [
            ('text_pipeline' , Pipeline([
                ('func1_get_text', func_transformer_get_text_features),
                ('func_print1', func_print_stuff),
                ('impute_text', func_transformer_imputer),
                
                ('encode1' , func_transformer_labelencoder),
                #('impute_text' , Imputer(strategy='most_frequent')), Do not use the inbuild imputer, since it requires human touch
                #('func_pd2np', func_transformer_convert_pd_np),
                ('encode2' , func_transformer_onehotencoder),
                ('func_print', func_print_stuff)
            ])
            ) 
            ,
            ('numeric_pipeline', Pipeline([
                ('func1_get_numeric', func_transformer_get_numeric_features),
                ('func_print1', func_print_stuff),
                ('impute_numeric', func_transformer_imputer),
                #('impute_numeric' , Imputer(strategy='median')),
                #('func_pd2np', func_transformer_convert_pd_np),
                ('scale', StandardScaler()),
                ('func_print', func_print_stuff)

            ])
            )
            ,
            ('engineer_pipeline_text', Pipeline([
                ('engineer_text', func_transformer_engineer_text),
                ('encode1' , func_transformer_labelencoder),
                ('encode2' , func_transformer_onehotencoder)
            ]
            )
            )
            ,
            ('engineer_pipeline_numeric', Pipeline([
                ('engineer_text', func_transformer_engineer_numeric),
             
            ]
            )
            )            
            
            
        ]
        )
    )
    #,
    #drop features manually
    #('feature_drop_manual', func_transformer_drop_manual)
])  



X.shape (1309, 11)
y.shape (1309,)


In [62]:
#Obtain the transformed feature matrix

X_transformed = pipe.fit_transform(X,y)
y_transformed = y



x.shape  (1309, 5)
x.shape  (1309, 2427)
x.shape  (1309, 6)
x.shape  (1309, 6)


In [63]:
print X_transformed[0:891].shape #
print X_transformed[891:].shape # 

(891, 3360)
(418, 3360)


In [64]:
#Train test split of transformed features
X_train,X_test, y_train,y_test = train_test_split(X_transformed[0:891],y_transformed[0:891], stratify = y_transformed[0:891])

print 'X_train.shape', X_train.shape
print 'X_test.shape', X_test.shape

#Model building
pipe1 = Pipeline([
    ('feature_union2', FeatureUnion(
            [
                ('pca', PCA())
#                 ,                
#                 ('pipe_feature_sel', Pipeline([
#                     ('variancethreshold', VarianceThreshold()),
#                     ('selectKbest', SelectKBest())
                    
#                 ]))
                ,
                ('select_from_model', SelectFromModel(estimator=RandomForestClassifier()))
 
            ]
        )
    )
    ,
    #('func_print', func_print_stuff),
    ('clf1', LogisticRegression())
    ,
    ('clf2', RandomForestClassifier())
])

param = {#'clf1__C': np.arange(.01,1,.01)  \
         #, 'feature_union2__pipe_feature_sel__selectKbest__k' : range(0,400,40) \
          'feature_union2__pca__n_components' : range(1,100,20) \
         #, 'feature_union2__select_from_model__estimator' : [RandomForestClassifier()] \
         , 'feature_union2__select_from_model__threshold' : ['median'] \
         , 'clf1' : [None] \
         , 'clf2__n_estimators' : range(1,50,5) 
         , 'clf2__criterion' : ['gini','entropy']
        , 'clf2__min_samples_split' : range(2,50,4)
        }

clf = GridSearchCV(pipe1, param_grid=param, cv=3)
clf.fit(X_train,y_train)


X_train.shape (668, 3360)
X_test.shape (223, 3360)


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('feature_union2', FeatureUnion(n_jobs=1,
       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('select_from_model', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_wei...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'feature_union2__pca__n_components': [1, 21, 41, 61, 81], 'clf2__n_estimators': [1, 6, 11, 16, 21, 26, 31, 36, 41, 46], 'clf2__criterion': ['gini', 'entropy'], 'clf1': [None], 'feature_union2__select_from_model__threshold': ['median'], 'clf2__min_samples_split': [2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [65]:
print clf.best_estimator_
print clf.best_score_
print clf.score(X_test,y_test)

#Last result
#0.832335329341
#0.825112107623

Pipeline(steps=[('feature_union2', FeatureUnion(n_jobs=1,
       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=41, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('select_from_model', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weigh...imators=31, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])
0.833832335329
0.820627802691


In [66]:

y_kaggle = clf.predict(X_transformed[891:])
pd.DataFrame(list(zip(X_kaggle.PassengerId,y_kaggle.astype(int))), columns=['PassengerId','Survived']).to_csv('/home/centos/My notebooks/Titanic dataset/output.csv', index=False)

In [67]:
print 'X_train.shape', X_train.shape
print 'X_test.shape', X_test.shape

X_train.shape (668, 3360)
X_test.shape (223, 3360)


#print y_kaggle.shape
#print X_transformed[891:].shape
#list(zip(X_kaggle.PassengerId,y_kaggle.astype(int)))
### v0.8 got me score of 0.76555
### v1.0 scored  0.78947
### v1.1 scored 0.80861